In [1]:
!nvidia-smi

Thu Jul 13 09:13:56 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 520.61.05    Driver Version: 520.61.05    CUDA Version: 11.8     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  On   | 00000000:89:00.0 Off |                    0 |
| N/A   37C    P0    57W / 300W |   1427MiB / 32768MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
!module load cuda/11.4.4

In [3]:
from pynvml import *
import torch.optim as optim


def print_gpu_utilization():
    nvmlInit()
    handle = nvmlDeviceGetHandleByIndex(0)
    info = nvmlDeviceGetMemoryInfo(handle)
    print(f"GPU memory occupied: {info.used//1024**2} MB.")


def print_summary(result):
    print(f"Time: {result.metrics['train_runtime']:.2f}")
    print(f"Samples/second: {result.metrics['train_samples_per_second']:.2f}")
    print_gpu_utilization()
    


In [4]:
print_gpu_utilization()

GPU memory occupied: 1684 MB.


In [5]:
# !pip -q datasets

In [6]:
model_name = "t5-small"
from transformers import AutoTokenizer, T5ForConditionalGeneration

[2023-07-13 09:14:12,702] [INFO] [real_accelerator.py:110:get_accelerator] Setting ds_accelerator to cuda (auto detect)


2023-07-13 09:14:19.580805: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-07-13 09:14:19.865448: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-07-13 09:14:25.346993: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-07-13 09:14:25.347216: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such 

In [7]:
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = T5ForConditionalGeneration.from_pretrained(model_name)


In [8]:
from datasets import load_dataset

In [9]:
import pandas as pd
from sklearn.model_selection import train_test_split
from datasets import Dataset




In [10]:
df1 = pd.read_csv('Training_set_IMDB/training_set_sample_no_target.csv')
df1 = df1.dropna()


shuffled_df = df1.sample(frac=1, random_state=42)  # Set random_state for reproducibility

train_df, eval_df = train_test_split(shuffled_df, test_size=0.1, random_state=42)

# 'train_df' will contain 80% of the shuffled data for training
# 'eval_df' will contain 20% of the shuffled data for evaluation


text = []
sql = []

for index, row in train_df.iterrows():
    
    text_input = "Translate to SQL: " + row['Text'],
    sql_input = row['SQL']
    text.append(text_input)
    sql.append(sql_input)
    
inputs = {"inputs": text,
        "target": sql}

train_dataset = Dataset.from_dict(inputs)

In [11]:
text_2 = []
sql_2 = []

for index, row in eval_df.iterrows():
    
    text_input = "Translate to SQL: " + row['Text'],
    sql_input = row['SQL']
    text_2.append(text_input)
    sql_2.append(sql_input)
    
inputs_2 = {"inputs": text_2,
        "target": sql_2}
eval_dataset = Dataset.from_dict(inputs_2)

In [12]:
def map_to_lenght(x):
    x["input_len"] = len(tokenizer(x["inputs"]).input_ids)
    x["input_longer_256"] = int(x["input_len"]>256)
    x["input_longer_128"] = int(x["input_len"]>128)
    x["input_longet_64"] = int(x["input_len"]>64)
    x["output_len"] = len(tokenizer(x["target"]).input_ids)
    x["output_longet_256"] = int(x["output_len"]>256)
    x["output_longet_128"] = int(x["output_len"]>128)
    x["output_longet_64"] = int(x["output_len"]>64)
    return x

sample_size = 10000
data_stats = train_dataset.select(range(sample_size)).map(map_to_lenght, num_proc=4)

Map (num_proc=4):   0%|          | 0/10000 [00:00<?, ? examples/s]

In [13]:
def compute_and_print(x):
    if len(x["input_len"])==sample_size:
        print(
            f"Input mean: {sum(x['input_len'])/sample_size} \n % of input len > 256: {sum(x['input_longer_256'])/sample_size}, \n % of input len > 128: {sum(x['input_longer_128'])/sample_size}, \n % of input len > 64: {sum(x['input_longet_64'])/sample_size}, \n Ouput mean: {sum(x['output_len'])/sample_size},\n% of output len > 256: {sum(x['output_longet_256'])/sample_size}, \n% of output len > 128: {sum(x['output_longet_128'])/sample_size}, \n% of output len > 64: {sum(x['output_longet_64'])/sample_size}")

output = data_stats.map(compute_and_print, batched=True, batch_size=-1)

Map:   0%|          | 0/10000 [00:00<?, ? examples/s]

Input mean: 1.0 
 % of input len > 256: 0.0, 
 % of input len > 128: 0.0, 
 % of input len > 64: 0.0, 
 Ouput mean: 106.4648,
% of output len > 256: 0.0, 
% of output len > 128: 0.2934, 
% of output len > 64: 0.7241


In [14]:
def convert_to_features(example_batch, padding = "max_length"):
    inputs = tokenizer.batch_encode_plus(example_batch["inputs"],is_split_into_words=True, max_length=64, truncation=True)
    
    targets = tokenizer.batch_encode_plus(example_batch["target"], max_length=256,truncation = True)
    if padding == "max_length":
        targets["inputs_ids"] = [
            [(l if l != tokenizer.pad_token_id else -100) for l in target] for target in targets["input_ids"]
        ]
    
    inputs["labels"] = targets['input_ids']
    return inputs

train_data = train_dataset.map(convert_to_features, batched=True, remove_columns=train_dataset.column_names)
test_data = eval_dataset.map(convert_to_features, batched=True, remove_columns=eval_dataset.column_names)

Map:   0%|          | 0/38756 [00:00<?, ? examples/s]

Map:   0%|          | 0/4307 [00:00<?, ? examples/s]

In [15]:
test_data

Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 4307
})

In [16]:
columns = ['input_ids', 'attention_mask', 'labels']

train_data.set_format(type='torch', columns=columns)
test_data.set_format(type='torch', columns=columns)

In [17]:
len(train_data)

38756

In [18]:
from transformers import Seq2SeqTrainer, Seq2SeqTrainingArguments, DataCollatorForSeq2Seq
import os

output_dir = 'T5-fine-tuned-with-IMDB-wikisql'


In [19]:
training_args = Seq2SeqTrainingArguments(
    output_dir=output_dir,
    per_device_train_batch_size=16,
    num_train_epochs=10,
    per_device_eval_batch_size=16,
    predict_with_generate=True,
    evaluation_strategy="epoch",
    do_train=True,
    learning_rate=2e-4,
    weight_decay=0.1,
    do_eval=True,
    logging_strategy="epoch",
    save_strategy="epoch",
    overwrite_output_dir=True,
    save_total_limit=3,
    load_best_model_at_end=True,
    push_to_hub=False,
    fp16=True 
)

In [20]:
! pip install -q rouge_score


In [21]:
from evaluate import load
rouge = load("rouge")

In [22]:
import numpy as np

def compute_metrics(pred):
    predictions, labels = pred
    
    pred_str = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    label_str = tokenizer.batch_decode(labels, skip_special_tokens=True)
    
    rouge_output = rouge.compute(predictions=pred_str, references=label_str)
    return {
        "rouge1": round(rouge_output['rouge1']* 100,4),
        "rouge2": round(rouge_output['rouge2']*100, 4),
        "rougeL": round(rouge_output['rougeL']*100, 4)
    }

In [23]:
test_data

data_collator = DataCollatorForSeq2Seq(tokenizer, model=model, label_pad_token_id=-100)

In [24]:
trainer = Seq2SeqTrainer(
    model = model,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=train_data,
    eval_dataset= test_data,
    tokenizer=tokenizer,
    data_collator=data_collator
)

In [25]:
trainer.train()

/home/toibazd/miniconda3/lib/python3.9/site-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
You're using a T5TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel
1,0.110700,0.016623,39.031700,36.486400,39.015200
2,0.022000,0.011903,39.032400,36.489200,39.015100
3,0.016000,0.009470,39.033700,36.489900,39.015800
4,0.012700,0.008118,39.033700,36.489900,39.017200
5,0.010400,0.007531,39.033000,36.489900,39.015800
6,0.008800,0.006945,39.032400,36.489200,39.015100
7,0.007500,0.006670,39.033000,36.489900,39.015800
8,0.006700,0.006512,39.033800,36.490200,39.017400
9,0.006000,0.006522,39.033800,36.490200,39.017400
10,0.005600,0.006458,39.032400,36.489200,39.015100


TrainOutput(global_step=24230, training_loss=0.02062915694944828, metrics={'train_runtime': 7669.4759, 'train_samples_per_second': 50.533, 'train_steps_per_second': 3.159, 'total_flos': 2671447320428544.0, 'train_loss': 0.02062915694944828, 'epoch': 10.0})

In [26]:
trainer.save_model(output_dir)

In [27]:
tokenizer.save_pretrained(output_dir)

('T5-fine-tuned-with-IMDB-wikisql/tokenizer_config.json',
 'T5-fine-tuned-with-IMDB-wikisql/special_tokens_map.json',
 'T5-fine-tuned-with-IMDB-wikisql/tokenizer.json')

In [28]:
model.eval()

T5ForConditionalGeneration(
  (shared): Embedding(32128, 512)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32128, 512)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=512, out_features=512, bias=False)
              (k): Linear(in_features=512, out_features=512, bias=False)
              (v): Linear(in_features=512, out_features=512, bias=False)
              (o): Linear(in_features=512, out_features=512, bias=False)
              (relative_attention_bias): Embedding(32, 8)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseActDense(
              (wi): Linear(in_features=512, out_features=2048, bias=False)
              (wo): Linear(in_features=2048, out_features=512, bias=False)
              (dropout): Drop

In [36]:
from transformers import TextStreamer
streamer = TextStreamer(tokenizer)
text = "Translate to SQL: how many films by female actors"

input = tokenizer(text, truncation=True, padding = "max_length", max_length=64, return_tensors="pt").to('cuda')

In [37]:
output = model.generate(**input, max_new_tokens = 256, streamer=streamer)
print(tokenizer.decode(output[0]))

<pad> SELECT d.id AS director_id, d.first_name, d.last_name, COUNT(m.id) AS movie_count FROM actors a JOIN roles r ON a.id = r.actor_id JOIN movies m ON r.movie_id = m.id where d.first_name = 'Federal' and d.last_name = 'Actor_id INNER JOIN movies m ON m.id, d.first_name, d.last_name</s>
<pad> SELECT d.id AS director_id, d.first_name, d.last_name, COUNT(m.id) AS movie_count FROM actors a JOIN roles r ON a.id = r.actor_id JOIN movies m ON r.movie_id = m.id where d.first_name = 'Federal' and d.last_name = 'Actor_id INNER JOIN movies m ON m.id, d.first_name, d.last_name</s>
